In [116]:
import scipy.io as sio
import numpy as np
import os
import sys
from PIL import Image
import errno
from sklearn.decomposition import PCA


In [2]:
mat_file = "dataset/PIE dataset/Pose05_64x64.mat"

data = sio.loadmat(mat_file)

data

{'__globals__': [],
 '__header__': 'MATLAB 5.0 MAT-file Platform: nt, Created on: Mon May 08 13:10:30 2017',
 '__version__': '1.0',
 'fea': array([[ 8,  9,  9, ..., 35, 36, 42],
        [ 6,  7,  7, ..., 38, 32, 31],
        [ 8,  8,  6, ..., 26, 25, 23],
        ..., 
        [11, 10,  8, ..., 36, 40, 42],
        [ 9, 11,  9, ..., 29, 31, 36],
        [12,  9,  9, ..., 29, 34, 38]], dtype=uint8),
 'gnd': array([[ 1],
        [ 1],
        [ 1],
        ..., 
        [68],
        [68],
        [68]], dtype=uint8),
 'isTest': array([[ 0.],
        [ 0.],
        [ 0.],
        ..., 
        [ 1.],
        [ 0.],
        [ 0.]])}

In [3]:
data['fea'].shape

(3332, 4096)

In [4]:
data['gnd'].shape

(3332, 1)

In [32]:
def consine_distance(feature1, feature2):
    '''
    @brief: compute consine distance between two vectors
    @param : feature1
    @param : feature2 
    '''
    num = np.dot(feature1,feature2.T)
    denom = np.linalg.norm(feature1) * np.linalg.norm(feature2)
    cos = num / denom #compute cos
    sim = 0.5 + 0.5 * float(cos)
    return sim

# ## test 
# a = np.array([1,1])
# b = np.array([-1,-1])
# print consine_distance(a,b)
        

1.11022302463e-16


In [146]:
def read_images ( path , sz = None ):
    c = 0
    X_train ,y_train = [] , []
    X_test ,y_test = [] , []
    for dirname , dirnames , filenames in os.walk ( path ):
        for subdirname in dirnames :
            subject_path = os.path.join ( dirname , subdirname )
            
            for filename in os.listdir ( subject_path ):
                if (filename != 'Thumbs.db'):
                    im = Image.open ( os.path.join ( subject_path , filename ))
                    im = im.convert ("L")
                    # resize to given size (if given )
                    if ( sz is not None ) :
                        im = im.resize (sz , Image.ANTIALIAS )
                        if (filename != '4.BMP' and filename != '8.BMP'):
                            X_train.append ( np.asarray (im , dtype = np.uint8 ) )
                            y_train.append (c)
                        else:
                            X_test.append ( np.asarray (im , dtype = np.uint8 ) )
                            y_test.append (c)
            c = c + 1
    return X_train , y_train, X_test, y_test

In [147]:
ROOT = "/Users/caishaofan/Desktop/IPython/dataset/ORL/ORL92112/bmp"
size = 23,28
X_train, y_train, X_test, y_test= read_images(ROOT, size)

In [148]:

X_train =  np.array(X_train)
y_train = np.array(y_train)
X_train_num = X_train.shape[0]

X_test = np.array(X_test)
y_test = np.array(y_test)
X_test_num = X_test.shape[0]
## reshape X_train X_test y_train y_test
X_train = X_train.reshape((X_train_num, -1))
X_test = X_test.reshape((X_test_num, -1))

print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(320, 644)
(80, 644)
(320,)
(80,)


In [154]:
pca = PCA(n_components = 150)

#transform the training data
X_train_reduced = pca.fit_transform(X_train)
#transform the test data
X_test_reduced = pca.transform(X_test)

print X_train_reduced.shape
print X_test_reduced.shape

(320, 150)
(80, 150)


In [155]:
print pca.explained_variance_ratio_

[ 0.19720308  0.14382384  0.07760078  0.06237401  0.05595871  0.03795961
  0.0280227   0.02611307  0.02166097  0.02014604  0.01609794  0.01565181
  0.01209159  0.01158073  0.01081075  0.00994615  0.00969912  0.00923013
  0.00825681  0.00755489  0.0071716   0.00703444  0.00644305  0.00587916
  0.00573107  0.00530108  0.00511901  0.00500038  0.004667    0.00418169
  0.00408531  0.00401289  0.00398561  0.0037448   0.00369534  0.00358233
  0.00344409  0.00336669  0.00323119  0.00305507  0.00296181  0.00288854
  0.00281789  0.00270893  0.00260537  0.00244632  0.00241921  0.00232452
  0.00230425  0.00223224  0.00215729  0.00207728  0.00199007  0.00189431
  0.00186434  0.00183232  0.00177024  0.00175006  0.00168607  0.00164604
  0.00163159  0.00160122  0.00153297  0.00149273  0.00148765  0.00141579
  0.00137787  0.00137434  0.00133318  0.00129154  0.00127032  0.00124361
  0.00119575  0.00118235  0.00115543  0.00113389  0.00110539  0.00107603
  0.00105944  0.00101659  0.0010115   0.0010018   0

In [156]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression()

#train
model.fit(X_train_reduced, y_train)
predictions = model.predict(X_test_reduced)

print classification_report(y_test, predictions)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00         2
          1       1.00      1.00      1.00         2
          2       0.67      1.00      0.80         2
          3       1.00      1.00      1.00         2
          4       0.50      1.00      0.67         2
          5       1.00      1.00      1.00         2
          6       1.00      1.00      1.00         2
          7       1.00      0.50      0.67         2
          8       1.00      1.00      1.00         2
          9       1.00      1.00      1.00         2
         10       1.00      1.00      1.00         2
         11       1.00      1.00      1.00         2
         12       1.00      1.00      1.00         2
         13       1.00      1.00      1.00         2
         14       1.00      0.50      0.67         2
         15       0.50      1.00      0.67         2
         16       1.00      1.00      1.00         2
         17       1.00      1.00      1.00   

In [151]:
print y_train

[ 0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  3
  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  6  6
  6  6  6  6  6  6  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  9  9  9
  9  9  9  9  9 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 12 12 12 12
 12 12 12 12 13 13 13 13 13 13 13 13 14 14 14 14 14 14 14 14 15 15 15 15 15
 15 15 15 16 16 16 16 16 16 16 16 17 17 17 17 17 17 17 17 18 18 18 18 18 18
 18 18 19 19 19 19 19 19 19 19 20 20 20 20 20 20 20 20 21 21 21 21 21 21 21
 21 22 22 22 22 22 22 22 22 23 23 23 23 23 23 23 23 24 24 24 24 24 24 24 24
 25 25 25 25 25 25 25 25 26 26 26 26 26 26 26 26 27 27 27 27 27 27 27 27 28
 28 28 28 28 28 28 28 29 29 29 29 29 29 29 29 30 30 30 30 30 30 30 30 31 31
 31 31 31 31 31 31 32 32 32 32 32 32 32 32 33 33 33 33 33 33 33 33 34 34 34
 34 34 34 34 34 35 35 35 35 35 35 35 35 36 36 36 36 36 36 36 36 37 37 37 37
 37 37 37 37 38 38 38 38 38 38 38 38 39 39 39 39 39 39 39 39]
